# Experiments 

Paper: Facts Fade Fast: Evaluating Memorization of Outdated Medical Knowledge in Large Language Models
Reproducibility task:

Read the paper: https://arxiv.org/pdf/2509.04304

Reproduce parts of their experiments investigating when LLMs predict outdated medical knowledge using this repository: https://github.com/jvladika/MedChange

Reproducibility target: Your goal is to reproduce parts of the result in Table 2. Specifically, you **evaluate the Qwen 2.5 7B and the BioMistral 7B model** on the **Changed Knowledge Dataset**. Your task is to reproduce the precision, recall F1colums for (b) outdated Lab. and (c) Latest Labels, as well as the F1 diff colum for those two LLMs.


## Imports

In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import re
from pathlib import Path

## Set up dataframes

In [2]:
# Load the scraped studies' data and extract all DOIs from the date string
# Studies with multiple iterations have the same DOI, but end with "v2", "v3", etc.

df = pd.read_csv("./FullScraped.csv", index_col=0)
dois = [d.split("doi: ")[1].strip() if "doi: " in d else "" for d in df.date.tolist()]


In [3]:
#Load the generated questions and labels from the text fiel
with open("generated_questions_labels.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

questions = list()
labels = list()

for idx,line in enumerate(lines):
    #if idx in missing_indices:
    #    continue
    line = line.split("\n")[0]
    line = line.split(" ||| ")
    questions.append(line[0].split("QUESTION: ")[1].strip())
    labels.append(line[1].split("LABEL: ")[1].strip())
    


In [4]:
# Group the questions by DOI, so that multiple iterations of the same study are together in a group
doigroups = list()
indices = {}
for idx in range(len(dois)):
    d = dois[idx]
    if d not in indices:
        indices[d] = idx 

doisorted = sorted(dois)

# Iterate through the sorted DOIs and group them by common prefix. "Pivot" is the earliest study in the group.
idx = 0
while idx < len(dois):
    pivot = doisorted[idx]
    if pivot == "":
        idx += 1
        continue
    doigroup = list()
    while idx < len(dois) and doisorted[idx].startswith(pivot):
        doigroup.append(indices[doisorted[idx]])
        idx += 1
    doigroups.append(doigroup)

doigroups = [list(set(dg)) for dg in doigroups]
doigroups = sorted(doigroups)

def remove_duplicates(lst):
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

doigroups = [remove_duplicates(dg) for dg in doigroups]
doigroups = sorted(doigroups)

In [5]:
# Create a mapping of study iterations (by index) to their labels, for studies with multiple iterations
long_doigroups = list()
long_indices = list()
for idx,dg in enumerate(doigroups):
    if len(dg) > 1:
        long_doigroups.append(dg)
        long_indices.append(idx)

verdict_map = list()
for ld in long_doigroups:
    instance_verdicts = {}
    sorted_ld = sorted(ld)
    for idx in sorted_ld:
        instance_verdicts[idx] = labels[idx]
    verdict_map.append(instance_verdicts)

print(len(verdict_map))
#verdict_map

1535


In [19]:
## Generated answers of all models.

filenames = [#'llama33-70b_answers.txt',
 #'mistral-24b_answers.txt',
 #'gpt4o-mini_answers.txt',
 'qwen25-7b_answers.txt',
 #'deepsek-v3_answers.txt',
 #'olmo_gguf_answers_13b.txt',
 #'pmcllama_answers.txt',
 'biomistral_answers.txt', 
 #'biomistral_answers_repro.txt', 
 #'qwen_answers_repro.txt'
 #'biomistral_answers_with_refuted.txt'
 ]
filenames_repro = [
 'biomistral_answers_repro.txt', 
 'qwen_answers_repro.txt',
 'biomistral_answers_with_refuted.txt', 
 'biomistral_original_template_answers.txt', 
 'biomistral_origianl_plain_answers.txt'

 ]
filenames = ["GeneratedAnswers/"+f for f in filenames]
filenames_repro = ["GeneratedAnswers/Replicability_models/"+f for f in filenames_repro]


In [11]:
#filenames_repro.append("../HPC_results/biomistral_answers_with_refuted.txt")
for file in filenames_repro:
    print(file)


GeneratedAnswers/Replicability_models/biomistral_answers_repro.txt
GeneratedAnswers/Replicability_models/qwen_answers_repro.txt
GeneratedAnswers/Replicability_models/biomistral_answers_with_refuted_repro.txt
GeneratedAnswers/Replicability_models/biomistral_original_template_answers.txt
GeneratedAnswers/Replicability_models/biomistral_origianl_plain_answers.txt


## Get MedChangeQA results (change knowledge subset)

In [12]:
## Get all the labels of MedChangeQA

final_labels = list()
final_keys = list()
outdated_keys = list()
outdated_labels = list()  
newest_labels = list()   
newest_keys = list()

for verdict_dict in verdict_map:
    final_label = verdict_dict[list(verdict_dict.keys())[0]]
    final_labels.append(final_label)
    final_keys.append(list(verdict_dict.keys())[0])

    verdict_changed = False
    for key in verdict_dict.keys():
        if verdict_dict[key] != final_label:

            outdated_keys.append(key)
            outdated_labels.append(verdict_dict[key])

            newest_labels.append(verdict_dict[list(verdict_dict.keys())[0]])
            newest_keys.append(list(verdict_dict.keys())[0])

            verdict_changed = True
            break

# MedChangeQA is a subset where 512 questions changed their verdict over time
len(final_labels), len(newest_labels) 

(1535, 512)

In [29]:

## Get final evaluation scores of the predicted LLM answers

def get_predictions_change_knowledge(filename, label_type, old=True):
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.readlines()
        #print('open')
        
    if ("biomistral" in filename or "pmcllama" in filename)and old is True:
        #print("biomistral or pcmllama")
        lines = lines[1::3]
    
    # Get all the predicted labels on the entire dataset MedRevQA
    llm_predicted_labels = list()
    tokens_supported = ("SUPPORTED", "Supported:", "EFFECTIVE",
                        ": Supported","(supported)", "(Supported)", "Effective:",
                         ": YES", "YES:", "POSITIVE", "(YES)", "SUPPORTed")
    
    tokens_nei =("NOT ENOUGH INFORMATION:", "NOT ENOUGH INFORMATION", "NOT_ENOUGH_INFORMATION",
                "Not Enough Information:", "Not enough information." )


    for line in lines:
        #line = re.sub(r'[^\w\s]','',line_notnorm).strip()
        
        if "REFUTED" in line or "REF UTED" in line:
            llm_predicted_labels.append("REFUTED")
        elif any(tok in line for tok in tokens_nei):
            llm_predicted_labels.append("NOT ENOUGH INFORMATION")
        elif any(tok in line for tok in tokens_supported):
            llm_predicted_labels.append("SUPPORTED")
        else:
            llm_predicted_labels.append("NO LABEL FOUND")
            #print(line)
            #print(line_notnorm)
            #print('.....')
        
    #print(f'len llm predicted labels: {len(llm_predicted_labels)}')
    #print(f'len "NO LABEL FOUND": {llm_predicted_labels.count('NO LABEL FOUND')}')
    print(f'REFUTED: {llm_predicted_labels.count("REFUTED")}')
    print(f'NOT ENOUGH INFORMATION: {llm_predicted_labels.count("NOT ENOUGH INFORMATION")}')
    print(f'SUPPORTED: {llm_predicted_labels.count("SUPPORTED")}')


    # Get only the changed-knowledge subset (MedChangeQA)
    #llm_predicted_medchange = np.array(llm_predicted_labels)[np.array(outdated_keys)]
    #llm_predicted_medchange = np.array(llm_predicted_labels, dtype=object)[np.array(outdated_keys, dtype=int)]

    mapper = {"SUPPORTED": 0, "REFUTED": 2, "NOT ENOUGH INFORMATION": 1}
        
    pred_all = np.array(llm_predicted_labels, dtype=object)[np.array(outdated_keys, dtype=int)]

    gold_all = np.array([mapper[g] for g in (outdated_labels if label_type=="OUTDATED" else newest_labels)], dtype=int)

    #print(f'len(pred_all): {len(pred_all)}')
    #print(f'len(gold_all): {len(gold_all)}')


    common_len = min(len(pred_all), len(gold_all))
    if len(pred_all) != len(gold_all):
        print(f"[LENGTH MISMATCH] pred={len(pred_all)} vs gold={len(gold_all)} → {common_len}")
    pred_slice = pred_all[:common_len]
    gold_slice = gold_all[:common_len]

    #  Mask  (discart 'NO LABEL FOUND')
    mask = (pred_slice != "NO LABEL FOUND")
    dropped = int((~mask).sum())
    if dropped:
        masked_pos = np.flatnonzero(~mask)

        subset_keys = np.array(outdated_keys, dtype=int)[:common_len]
        masked_items = subset_keys[masked_pos]

        is_triplet_used = ("biomistral" in filename) or ("pmcllama" in filename)

        print(f"[NO LABEL FOUND] {dropped} points masked. Showing up to 50:")
        for i, pos in enumerate(masked_pos[:50], start=1):
            item_idx = int(masked_items[i-1])  
            if 0 <= item_idx < len(lines):
                preview = lines[item_idx].rstrip("\n")
            else:
                preview = "<out of range in current 'lines' view>"

            if is_triplet_used:
                file_line = item_idx * 3 + 2  
            else:
                file_line = item_idx + 1

            if len(preview) > 220:
                preview = preview[:220] + "…"
            print(f"  - subset_pos={int(pos)} | item_idx={item_idx} | file_line={file_line} | {preview}")

    
    y_test = gold_slice[mask]    
    y_pred = np.array([mapper[p] for p in pred_slice[mask]], dtype=int)

    # Calculate precision, recall, and F1 score with macro averaging
    precision_macro = precision_score(y_test, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)

    classes = np.unique(y_test)
    present = np.unique(y_pred)
    missing_preds = set(classes) - set(present)
    if missing_preds:
        print("[INFO] No info for classes:", missing_preds)

    ## get precision, recall, f1, accuracy rounded to 4 decimal places

    print(f"Precision: {precision_macro:.4f} | Recall: {recall_macro:.4f} | F1: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

    #WE ADDED THIS RETURN 
    return precision_macro,recall_macro, f1_macro


In [38]:
metrics = ['model', 'P_outdated', 'R_outdated', 'F1_outdated', 'P_latest', 'R_latest', 'F1_latest', 'F1_diff' ]
df = pd.DataFrame(columns=metrics)

In [39]:

summary_rows = []
print('ORIGINAL PAPER RESULTS')
for file in filenames: 
    pat = re.compile(r'([^/\\]+)\.txt$', re.IGNORECASE)
    name_re = pat.search(file).group(1) 
    print('-----')
    print(file)
    print('-----')
    print(" Latest")
    p_latest, r_latest, f1_latest = get_predictions_change_knowledge(file, label_type='NEWEST', old=True)
    print(" Outdated")
    p_outdated, r_outdated, f1_outdated = get_predictions_change_knowledge(file, label_type='OUTDATED', old=True)
    print('')
    f1_diff = (f1_latest - f1_outdated)*100
    print(f"F1 diff: {f1_diff}")
    print('')

    df.loc[len(df)] = {
        'model':      name_re,
        'P_outdated': round(p_outdated * 100, 1),
        'R_outdated': round(r_outdated * 100, 1),
        'F1_outdated': round(f1_outdated * 100, 1),
        'P_latest':   round(p_latest * 100, 1),
        'R_latest':   round(r_latest * 100, 1),
        'F1_latest':  round(f1_latest * 100, 1),
        'F1_diff':    round(f1_diff, 1)
    }

    original_diff = f1_diff
    
    print(file.split("/")[-1][:4])
    # save partial results
    summary_rows.append({
        "file": file.split("/")[-1][:4],
        "orig_latest": round(f1_latest*100,1),
        "orig_outdated": round(f1_outdated*100,1),
        "orig_diff": round(original_diff,1),
        "our_latest": None,
        "our_outdated": None,
        "our_diff": None
    })
    



ORIGINAL PAPER RESULTS
-----
GeneratedAnswers/qwen25-7b_answers.txt
-----
 Latest
REFUTED: 2213
NOT ENOUGH INFORMATION: 2261
SUPPORTED: 12027
Precision: 0.2712 | Recall: 0.3082 | F1: 0.2602
Accuracy: 0.3730
 Outdated
REFUTED: 2213
NOT ENOUGH INFORMATION: 2261
SUPPORTED: 12027
Precision: 0.4262 | Recall: 0.3711 | F1: 0.3076
Accuracy: 0.3535

F1 diff: -4.739933696312237

qwen
-----
GeneratedAnswers/biomistral_answers.txt
-----
 Latest
REFUTED: 4605
NOT ENOUGH INFORMATION: 5078
SUPPORTED: 6073
[NO LABEL FOUND] 20 points masked. Showing up to 50:
  - subset_pos=41 | item_idx=8141 | file_line=24425 | Answer: 1. Instrumental dysphagia (swallowing of solids) |||             2. Oral dysphagia (swallowing of liquids) |||             3. Nutritional support (enteral feeding) |||             INITIAL STATEMENT: All of the ab…
  - subset_pos=78 | item_idx=4512 | file_line=13538 | Answer: 1. Intravenous gadolinium-based contrast agents (GBCAs) in patients with advanced renal insufficiency.
  - subset

In [40]:
print('OUR MODEL RESULTS')
for file in filenames_repro: 
    pat = re.compile(r'([^/\\]+)\.txt$', re.IGNORECASE)
    name_re = pat.search(file).group(1) 
    print('-----')
    print(file)
    print('-----')
    print(" Latest")
    p_latest, r_latest, f1_latest = get_predictions_change_knowledge(file, label_type='NEWEST', old=False)
    print(" Outdated")
    p_outdated, r_outdated, f1_outdated = get_predictions_change_knowledge(file, label_type='OUTDATED', old=False)
    print('')
    f1_diff = (f1_latest - f1_outdated)*100
    print(f"F1 diff: {f1_diff}")
    print('')

    df.loc[len(df)] = {
        'model':      name_re,
        'P_outdated': round(p_outdated * 100, 1),
        'R_outdated': round(r_outdated * 100, 1),
        'F1_outdated': round(f1_outdated * 100, 1),
        'P_latest':   round(p_latest * 100, 1),
        'R_latest':   round(r_latest * 100, 1),
        'F1_latest':  round(f1_latest * 100, 1),
        'F1_diff':    round(f1_diff, 1)
    }


    

    our_diff = f1_diff


    print( file.split("/")[-1][:4])
    # find matching original row (same file)
    for row in summary_rows:
        if row["file"] == file.split("/")[-1][:4]:
            row["our_latest"] = round(f1_latest*100,1)
            row["our_outdated"] = round(f1_outdated*100,1)
            row["our_diff"] = round(our_diff,1)
            break
    

OUR MODEL RESULTS
-----
GeneratedAnswers/Replicability_models/biomistral_answers_repro.txt
-----
 Latest
REFUTED: 0
NOT ENOUGH INFORMATION: 14391
SUPPORTED: 2110
[INFO] No info for classes: {2}
Precision: 0.2707 | Recall: 0.3359 | F1: 0.2363
Accuracy: 0.3359
 Outdated
REFUTED: 0
NOT ENOUGH INFORMATION: 14391
SUPPORTED: 2110
[INFO] No info for classes: {2}
Precision: 0.2587 | Recall: 0.3367 | F1: 0.2703
Accuracy: 0.4414

F1 diff: -3.399789103922485

biom
-----
GeneratedAnswers/Replicability_models/qwen_answers_repro.txt
-----
 Latest
REFUTED: 453
NOT ENOUGH INFORMATION: 4433
SUPPORTED: 11613
Precision: 0.3355 | Recall: 0.3133 | F1: 0.2681
Accuracy: 0.3789
 Outdated
REFUTED: 453
NOT ENOUGH INFORMATION: 4433
SUPPORTED: 11613
Precision: 0.3982 | Recall: 0.3803 | F1: 0.3092
Accuracy: 0.3887

F1 diff: -4.1095515898888655

qwen
-----
GeneratedAnswers/Replicability_models/biomistral_answers_with_refuted.txt
-----
 Latest
REFUTED: 141
NOT ENOUGH INFORMATION: 11666
SUPPORTED: 4644
[NO LABEL FOUN

In [42]:
df = df.sort_values(by='model')
df

,model,P_outdated,R_outdated,F1_outdated,P_latest,R_latest,F1_latest,F1_diff
1,biomistral_answers,34.5,35.3,34.1,35.7,35.9,35.7,1.7
2,biomistral_answers_repro,25.9,33.7,27.0,27.1,33.6,23.6,-3.4
4,biomistral_answers_with_refuted,61.0,36.3,31.7,26.3,33.9,26.6,-5.1
6,biomistral_origianl_plain_answers,38.1,36.4,32.5,38.6,33.6,27.7,-4.8
5,biomistral_original_template_answers,27.7,35.5,29.8,26.0,33.5,24.8,-5.0
0,qwen25-7b_answers,42.6,37.1,30.8,27.1,30.8,26.0,-4.7
3,qwen_answers_repro,39.8,38.0,30.9,33.5,31.3,26.8,-4.1


In [43]:
df_summary = pd.DataFrame(summary_rows)

print("\nSUMMARY TABLE")
df_summary




SUMMARY TABLE


,file,orig_latest,orig_outdated,orig_diff,our_latest,our_outdated,our_diff
0,qwen,26.0,30.8,-4.7,26.8,30.9,-4.1
1,biom,35.7,34.1,1.7,27.7,32.5,-4.8
